In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
# from PIL import Image
# import xml.etree.ElementTree as ET

In [2]:
#For model
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
#For shwing images and graphs
from matplotlib import pyplot as plt

from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
classes = ('mask_worn_incorrectly', 'with_mask', 'without_mask')

In [11]:
imagePath = '/content/drive/MyDrive/Colab Notebooks/257-01/Classes'
transform = transforms.Compose([transforms.Resize((128,128)), transforms.ToTensor()])#Just resizing the image to 128*128 size
data = datasets.ImageFolder(root=imagePath, transform=transform)
loader = DataLoader(dataset=data,batch_size=32)

# New Section